In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime as dt
from sklearn.metrics import *

In [2]:
games = pd.read_csv('input/games.csv')

In [3]:
teams = pd.read_csv('input/teams.csv')

In [4]:
players = pd.read_csv('input/players.csv')

In [5]:
ranking = pd.read_csv('input/ranking.csv')

In [6]:
salaries = pd.read_csv('input/salaries.csv')

In [7]:
games = games.drop(columns = ["TEAM_ID_home","TEAM_ID_away","GAME_STATUS_TEXT"])

In [8]:
games = games.rename(columns={'GAME_DATE_EST':'FECHA',
                              'HOME_TEAM_ID':'EQUIPO_LOCAL_ID',
                              'VISITOR_TEAM_ID':'EQUIPO_VISITANTE_ID',
                              'SEASON':'TEMPORADA',
                              'PTS_home':'PTS_LOCAL',
                              'FG_PCT_home':'%TC_LOCAL',
                              'FT_PCT_home':'%TL_LOCAL',
                              'FG3_PCT_home':'%3P_LOCAL',
                              'AST_home':'AST_LOCAL',
                              'REB_home':'REB_LOCAL',
                              'PTS_away':'PTS_VISITANTE',
                              'FG_PCT_away':'%TC_VISITANTE',
                              'FT_PCT_away':'%TL_VISITANTE',
                              'FG3_PCT_away':'%3P_VISITANTE',
                              'AST_away':'AST_VISITANTE',
                              'REB_away':'REB_VISITANTE',
                              'HOME_TEAM_WINS':'VICTORIA'})

In [9]:
games.head()

,FECHA,GAME_ID,EQUIPO_LOCAL_ID,EQUIPO_VISITANTE_ID,TEMPORADA,PTS_LOCAL,%TC_LOCAL,%TL_LOCAL,%3P_LOCAL,AST_LOCAL,REB_LOCAL,PTS_VISITANTE,%TC_VISITANTE,%TL_VISITANTE,%3P_VISITANTE,AST_VISITANTE,REB_VISITANTE,VICTORIA
0,2020-03-01,21900895,1610612766,1610612749,2019,85.0,0.354,0.900,0.229,22.0,47.0,93.0,0.402,0.762,0.226,20.0,61.0,0
1,2020-03-01,21900896,1610612750,1610612742,2019,91.0,0.364,0.400,0.310,19.0,57.0,111.0,0.468,0.632,0.275,28.0,56.0,0
2,2020-03-01,21900897,1610612746,1610612755,2019,136.0,0.592,0.805,0.542,25.0,37.0,130.0,0.505,0.650,0.488,27.0,37.0,1
3,2020-03-01,21900898,1610612743,1610612761,2019,133.0,0.566,0.700,0.500,38.0,41.0,118.0,0.461,0.897,0.263,24.0,36.0,1
4,2020-03-01,21900899,1610612758,1610612765,2019,106.0,0.407,0.885,0.257,18.0,51.0,100.0,0.413,0.667,0.429,23.0,42.0,1


In [10]:
categoricalColumns = ['EQUIPO_LOCAL_ID', 'EQUIPO_VISITANTE_ID', 'GAME_ID','VICTORIA']
for i in categoricalColumns:
    games[i] = games[i].astype('category')


In [11]:
#games = pd.get_dummies(games, prefix=['VICTORIA'], columns=['VICTORIA'])

In [12]:
games['VICTORIA']= games['VICTORIA'].replace({0: 'VISITANTE', 1: 'LOCAL'})

In [14]:
games.head()

,FECHA,GAME_ID,EQUIPO_LOCAL_ID,EQUIPO_VISITANTE_ID,TEMPORADA,PTS_LOCAL,%TC_LOCAL,%TL_LOCAL,%3P_LOCAL,AST_LOCAL,REB_LOCAL,PTS_VISITANTE,%TC_VISITANTE,%TL_VISITANTE,%3P_VISITANTE,AST_VISITANTE,REB_VISITANTE,VICTORIA
0,2020-03-01,21900895,1610612766,1610612749,2019,85.0,0.354,0.900,0.229,22.0,47.0,93.0,0.402,0.762,0.226,20.0,61.0,VISITANTE
1,2020-03-01,21900896,1610612750,1610612742,2019,91.0,0.364,0.400,0.310,19.0,57.0,111.0,0.468,0.632,0.275,28.0,56.0,VISITANTE
2,2020-03-01,21900897,1610612746,1610612755,2019,136.0,0.592,0.805,0.542,25.0,37.0,130.0,0.505,0.650,0.488,27.0,37.0,LOCAL
3,2020-03-01,21900898,1610612743,1610612761,2019,133.0,0.566,0.700,0.500,38.0,41.0,118.0,0.461,0.897,0.263,24.0,36.0,LOCAL
4,2020-03-01,21900899,1610612758,1610612765,2019,106.0,0.407,0.885,0.257,18.0,51.0,100.0,0.413,0.667,0.429,23.0,42.0,LOCAL


In [15]:
#games = games.rename(columns={'VICTORIA_1':'VICTORIA_LOCAL',
#                             'VICTORIA_0':'VICTORIA_VISITANTE'
#                             })

In [16]:
#len(games.VICTORIA_VISITANTE)

In [17]:
#games['RESULTADO_V_L'] = games[['VICTORIA_VISITANTE','VICTORIA_LOCAL']].apply(lambda x: pd.Series([x.values]), axis=1)

In [18]:
#games = games.drop(columns = ["VICTORIA_VISITANTE","VICTORIA_LOCAL"])

In [19]:
#games.head()

In [20]:
games = games.dropna()

In [21]:
games.isnull().sum()

FECHA                  0
GAME_ID                0
EQUIPO_LOCAL_ID        0
EQUIPO_VISITANTE_ID    0
TEMPORADA              0
PTS_LOCAL              0
%TC_LOCAL              0
%TL_LOCAL              0
%3P_LOCAL              0
AST_LOCAL              0
REB_LOCAL              0
PTS_VISITANTE          0
%TC_VISITANTE          0
%TL_VISITANTE          0
%3P_VISITANTE          0
AST_VISITANTE          0
REB_VISITANTE          0
VICTORIA               0
dtype: int64

In [22]:
games.dtypes

FECHA                    object
GAME_ID                category
EQUIPO_LOCAL_ID        category
EQUIPO_VISITANTE_ID    category
TEMPORADA                 int64
PTS_LOCAL               float64
%TC_LOCAL               float64
%TL_LOCAL               float64
%3P_LOCAL               float64
AST_LOCAL               float64
REB_LOCAL               float64
PTS_VISITANTE           float64
%TC_VISITANTE           float64
%TL_VISITANTE           float64
%3P_VISITANTE           float64
AST_VISITANTE           float64
REB_VISITANTE           float64
VICTORIA                 object
dtype: object

In [23]:
games['FECHA'] = pd.to_datetime(games['FECHA'])
games['FECHA'] = games['FECHA'].apply(lambda x: x.toordinal())

In [24]:
games.head()

,FECHA,GAME_ID,EQUIPO_LOCAL_ID,EQUIPO_VISITANTE_ID,TEMPORADA,PTS_LOCAL,%TC_LOCAL,%TL_LOCAL,%3P_LOCAL,AST_LOCAL,REB_LOCAL,PTS_VISITANTE,%TC_VISITANTE,%TL_VISITANTE,%3P_VISITANTE,AST_VISITANTE,REB_VISITANTE,VICTORIA
0,737485,21900895,1610612766,1610612749,2019,85.0,0.354,0.900,0.229,22.0,47.0,93.0,0.402,0.762,0.226,20.0,61.0,VISITANTE
1,737485,21900896,1610612750,1610612742,2019,91.0,0.364,0.400,0.310,19.0,57.0,111.0,0.468,0.632,0.275,28.0,56.0,VISITANTE
2,737485,21900897,1610612746,1610612755,2019,136.0,0.592,0.805,0.542,25.0,37.0,130.0,0.505,0.650,0.488,27.0,37.0,LOCAL
3,737485,21900898,1610612743,1610612761,2019,133.0,0.566,0.700,0.500,38.0,41.0,118.0,0.461,0.897,0.263,24.0,36.0,LOCAL
4,737485,21900899,1610612758,1610612765,2019,106.0,0.407,0.885,0.257,18.0,51.0,100.0,0.413,0.667,0.429,23.0,42.0,LOCAL


In [25]:
y = games.VICTORIA

In [26]:
X = games.drop(columns =['VICTORIA','GAME_ID','EQUIPO_LOCAL_ID','EQUIPO_VISITANTE_ID'])

In [27]:
print(y.shape)
print(X.shape)

(23096,)
(23096, 14)


In [28]:
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline
pipeline = [
    StandardScaler(),
    Normalizer(),
]

tr = make_pipeline(*pipeline)

Xpr = tr.fit_transform(X)
Xpr = pd.DataFrame(Xpr,columns=X.columns)
print(Xpr.info())
Xpr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23096 entries, 0 to 23095
Data columns (total 14 columns):
FECHA            23096 non-null float64
TEMPORADA        23096 non-null float64
PTS_LOCAL        23096 non-null float64
%TC_LOCAL        23096 non-null float64
%TL_LOCAL        23096 non-null float64
%3P_LOCAL        23096 non-null float64
AST_LOCAL        23096 non-null float64
REB_LOCAL        23096 non-null float64
PTS_VISITANTE    23096 non-null float64
%TC_VISITANTE    23096 non-null float64
%TL_VISITANTE    23096 non-null float64
%3P_VISITANTE    23096 non-null float64
AST_VISITANTE    23096 non-null float64
REB_VISITANTE    23096 non-null float64
dtypes: float64(14)
memory usage: 2.5 MB
None


,FECHA,TEMPORADA,PTS_LOCAL,%TC_LOCAL,%TL_LOCAL,%3P_LOCAL,AST_LOCAL,REB_LOCAL,PTS_VISITANTE,%TC_VISITANTE,%TL_VISITANTE,%3P_VISITANTE,AST_VISITANTE,REB_VISITANTE
0,0.339946,0.334603,-0.263530,-0.369702,0.280676,-0.218625,-0.019709,0.114976,-0.094452,-0.164513,0.011172,-0.215775,-0.042927,0.584512
1,0.283661,0.279203,-0.142961,-0.279361,-0.589729,-0.065577,-0.112760,0.346720,0.151178,0.059149,-0.199187,-0.108153,0.224693,0.360388
2,0.297277,0.292605,0.454893,0.403275,0.081385,0.282020,0.083702,-0.162273,0.412856,0.177390,-0.178492,0.214185,0.201352,-0.129457
3,0.304025,0.299246,0.423989,0.331253,-0.102215,0.223483,0.532928,-0.058443,0.257893,0.041067,0.242064,-0.134788,0.101216,-0.159693
4,0.474494,0.467035,0.082598,-0.257773,0.350418,-0.237588,-0.242323,0.328279,0.017777,-0.174864,-0.239286,0.197060,0.103497,0.006388


In [29]:
from sklearn.model_selection import train_test_split

Xpr_train, Xpr_test, y_train, y_test = train_test_split(Xpr, y, test_size=0.2)

In [30]:
print(Xpr_train.shape)
print(Xpr_test.shape)
print(y_train.shape)
print(y_test.shape)

(18476, 14)
(4620, 14)
(18476,)
(4620,)


In [31]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=800, max_depth=20, min_samples_leaf=3, random_state=111, n_jobs = -1)

model.fit(Xpr_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=800,
                       n_jobs=-1, oob_score=False, random_state=111, verbose=0,
                       warm_start=False)

In [32]:
y_pred=model.predict(Xpr_test)
print('Accurancy', accuracy_score(y_test, y_pred))
print("Precision", precision_score(y_test, y_pred, average='weighted'))
print("Recall", recall_score(y_test, y_pred, average='weighted'))

Accurancy 0.9824675324675325
Precision 0.9825421973192685
Recall 0.9824675324675325


In [33]:
model.predict_proba(Xpr_test)

array([[0.94298408, 0.05701592],
       [0.89958333, 0.10041667],
       [0.99477679, 0.00522321],
       ...,
       [0.96133085, 0.03866915],
       [0.00141843, 0.99858157],
       [0.99504167, 0.00495833]])

In [37]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

y_score = model.predict_proba(Xpr_test)


display(pd.DataFrame({
    "real":y_pred,
    "predicted":y_test,
    "probaVisitante":y_score[:,1],
    "probaLocal":y_score[:,0]
}).head(30))


,real,predicted,probaVisitante,probaLocal
14122,LOCAL,LOCAL,0.057016,0.942984
18265,LOCAL,LOCAL,0.100417,0.899583
19202,LOCAL,LOCAL,0.005223,0.994777
12596,VISITANTE,VISITANTE,0.912624,0.087376
8266,VISITANTE,VISITANTE,0.990182,0.009818
23004,VISITANTE,VISITANTE,0.980819,0.019181
12683,LOCAL,LOCAL,0.048135,0.951865
11678,VISITANTE,VISITANTE,0.646025,0.353975
123,VISITANTE,VISITANTE,0.930994,0.069006
12657,LOCAL,LOCAL,0.275086,0.724914
